In [1]:
import os
import sys
import yaml
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F
import time
from torch.utils.data import DataLoader
from imblearn.under_sampling import RandomUnderSampler

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%pip install wandb -qU
%matplotlib inline

# Get the current working directory
notebook_dir = notebook_dir = os.path.dirname(os.path.abspath("__file__"))  
project_dir = os.path.abspath(os.path.join(notebook_dir, '..')) 
if project_dir not in sys.path:
    sys.path.append(project_dir)


from binary_classifier import (get_transforms, load_data, split_data, set_seeds, 
                 verify_splits, verify_data, plot_species_grid,
                 verify_loader_transforms)
from binary_classifier.data_utils import ImagesDataset
from binary_classifier.models import build_resnet50_basic, build_efficientnet_v2_basic
from binary_classifier.train import setup_training, evaluate, train

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Locate the YAML file relative to the notebook's location
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# You need to update this path to your new .yaml file
config_path = os.path.join(notebook_dir, "../configs/default_single_ensemble_training.yaml")

# Load the YAML file
with open(config_path, "r") as f:
    config = yaml.safe_load(f)
print(config)

{'device': 'cuda', 'model': {'num_classes': 1, 'architecture': 'resnet50', 'freeze_backbone': False, 'dropout': 0.1, 'hidden_units1': 100}, 'train': {'batch_size': 32, 'epochs': 10, 'lr': 0.001, 'momentum': 0.9, 'optimizer': 'adam', 'criterion': 'binary_cross_entropy'}, 'experiment': {'seed': 42, 'experiment_name': 'single_model_ensemble_train_resnet_rodent'}, 'transforms': {'resize': [224, 224], 'horizontal_flip': 0.5, 'rotate': 15, 'jitter': {'brightness': 0.2, 'contrast': 0.2, 'saturation': 0.2, 'hue': 0.1}, 'custom': {'block_timestamp': True}}, 'log': {'img_count': 50}}


In [4]:
print(torch.__version__)
print(torch.backends.mps.is_available())
device = config["device"]
print(f"Running on device: {device}")

2.5.1
False
Running on device: cuda


In [5]:
train_features, test_features, train_labels, species_labels = load_data()
#species_labels

In [6]:
train_labels_antelope_duiker = train_labels["antelope_duiker"]
# not_ant_duiker = (train_labels_antelope_duiker==0)*1.0
# not_ant_duiker.name = "not_antelope_duiker"
# train_labels_antelope_duiker = pd.concat([train_labels_antelope_duiker, not_ant_duiker], axis= 1)

train_labels_bird = train_labels["bird"]
# not_bird = (train_labels_antelope_duiker==0)*1.0
# not_bird.name = "not_bird"
# train_labels_bird = pd.concat([train_labels_bird, not_bird], axis= 1)

train_labels_blank = train_labels["blank"]
# not_blank = (train_labels_blank==0)*1.0
# not_blank.name = "not_blank"
# train_labels_blank = pd.concat([train_labels_blank, not_blank], axis= 1)

train_labels_civet_genet = train_labels["civet_genet"]
# not_civet_genet = (train_labels_civet_genet==0)*1.0
# not_civet_genet.name = "not_civet_genet"
# train_labels_civet_genet = pd.concat([train_labels_civet_genet, not_civet_genet], axis= 1)


train_labels_hog = train_labels["hog"]
# not_hog = (train_labels_hog==0)*1.0
# not_hog.name = "not_hog"
# train_labels_hog = pd.concat([train_labels_hog, not_hog], axis= 1)


train_labels_leopard = train_labels["leopard"]
# not_leopard = (train_labels_leopard==0)*1.0
# not_leopard.name = "not_leopard"
# train_labels_leopard = pd.concat([train_labels_leopard, not_leopard], axis= 1)


train_labels_monkey_prosimian = train_labels["monkey_prosimian"]
# not_monkey_prosimian = (train_labels_monkey_prosimian==0)*1.0
# not_monkey_prosimian.name = "not_monkey_prosimian"
# train_labels_monkey_prosimian = pd.concat([train_labels_monkey_prosimian, not_monkey_prosimian], axis= 1)

train_labels_rodent = train_labels["rodent"]
# not_rodent = (train_labels_rodent==0)*1.0
# not_rodent.name = "not_rodent"
# train_labels_rodent = pd.concat([train_labels_rodent, not_rodent], axis= 1)



In [7]:
# Get transforms
train_transforms, val_transforms = get_transforms(config)

In [8]:
set_seeds(config["experiment"]["seed"])

X_train_antelope_duiker, X_val_antelope_duiker, y_train_antelope_duiker, y_val_antelope_duiker = split_data(train_features, train_labels_antelope_duiker)
X_train_bird, X_val_bird, y_train_bird, y_val_bird = split_data(train_features, train_labels_bird)
X_train_blank, X_val_blank, y_train_blank, y_val_blank = split_data(train_features, train_labels_blank)
X_train_civet_genet, X_val_civet_genet, y_train_civet_genet, y_val_civet_genet = split_data(train_features, train_labels_civet_genet)
X_train_hog, X_val_hog, y_train_hog, y_val_hog = split_data(train_features, train_labels_hog)
X_train_leopard, X_val_leopard, y_train_leopard, y_val_leopard = split_data(train_features, train_labels_leopard)
X_train_monkey_prosimian, X_val_monkey_prosimian, y_train_monkey_prosimian, y_val_monkey_prosimian = split_data(train_features, train_labels_monkey_prosimian)
X_train_rodent, X_val_rodent, y_train_rodent, y_val_rodent = split_data(train_features, train_labels_rodent)

In [9]:
random_under_sampler = RandomUnderSampler(sampling_strategy=1)

X_train_antelope_duiker_resampled, y_train_antelope_duiker_resampled = random_under_sampler.fit_resample(X_train_antelope_duiker, y_train_antelope_duiker)

X_train_bird_resampled, y_train_bird_resampled = random_under_sampler.fit_resample(X_train_bird, y_train_bird)

X_train_blank_resampled, y_train_blank_resampled = random_under_sampler.fit_resample(X_train_blank, y_train_blank)

X_train_civet_genet_resampled, y_train_civet_genet_resampled = random_under_sampler.fit_resample(X_train_civet_genet, y_train_civet_genet)

X_train_hog_resampled, y_train_hog_resampled = random_under_sampler.fit_resample(X_train_hog, y_train_hog)

X_train_leopard_resampled, y_train_leopard_resampled = random_under_sampler.fit_resample(X_train_leopard, y_train_leopard)

X_train_monkey_prosimian_resampled, y_train_monkey_prosimian_resampled = random_under_sampler.fit_resample(X_train_monkey_prosimian, y_train_monkey_prosimian)

X_train_rodent_resampled, y_train_rodent_resampled = random_under_sampler.fit_resample(X_train_rodent, y_train_rodent)



In [ ]:
# Create datasets antelope

set_seeds(config["experiment"]["seed"])

# Create datasets antelope
train_dataset_antelope_duiker = ImagesDataset(
    features=X_train_antelope_duiker_resampled, 
    labels=y_train_antelope_duiker_resampled, 
    transform=train_transforms, 
    device=device)

val_dataset_antelope_duiker = ImagesDataset(
    features=X_val_antelope_duiker, 
    labels=y_val_antelope_duiker,
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_antelope_duiker = DataLoader(
    train_dataset_antelope_duiker, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True,
    pin_memory=False)

val_loader_antelope_duiker = DataLoader(
    val_dataset_antelope_duiker, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False,
    pin_memory=False)

# model_antelope_duiker = build_efficientnet_v2_basic(
#     num_classes = config["model"]["num_classes"],
#     hidden_units1 = config["model"]["hidden_units1"],
#     dropout = config["model"]["dropout"] 
# )

model_antelope_duiker = build_resnet50_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)

model_antelope_duiker = model_antelope_duiker.to(device)

In [ ]:
# Create datasets bird
train_dataset_bird = ImagesDataset(
    features=X_train_bird_resampled,
    labels=y_train_bird_resampled, 
    transform=train_transforms, 
    device=device)


val_dataset_bird = ImagesDataset(
    features=X_val_bird, 
    labels=y_val_bird, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_bird = DataLoader(
    train_dataset_bird, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=False)

val_loader_bird = DataLoader(
    val_dataset_bird, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=False)

model_bird = build_resnet50_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_bird = model_bird.to(device)

In [ ]:
# Create datasets blank
train_dataset_blank = ImagesDataset(
    features=X_train_blank_resampled, 
    labels=y_train_blank_resampled, 
    transform=train_transforms, 
    device=device)

val_dataset_blank = ImagesDataset(
    features=X_val_blank, 
    labels=y_val_blank, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_blank = DataLoader(
    train_dataset_blank, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=False)

val_loader_blank = DataLoader(
    val_dataset_blank, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=False)

model_blank = build_resnet50_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_blank = model_blank.to(device)

In [ ]:
# Create datasets civet genet
train_dataset_civet_genet = ImagesDataset(
    features=X_train_civet_genet_resampled, 
    labels=y_train_civet_genet_resampled, 
    transform=train_transforms, 
    device=device)

val_dataset_civet_genet = ImagesDataset(
    features=X_val_civet_genet, 
    labels=y_val_civet_genet, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_civet_genet = DataLoader(
    train_dataset_civet_genet, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=False)

val_loader_civet_genet = DataLoader(
    val_dataset_civet_genet, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=False)

model_civet_genet = build_resnet50_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_civet_genet = model_civet_genet.to(device)

In [ ]:
# Create datasets hog
train_dataset_hog = ImagesDataset(
    features=X_train_hog_resampled, 
    labels=y_train_hog_resampled, 
    transform=train_transforms, 
    device=device)

val_dataset_hog = ImagesDataset(
    features=X_val_hog, 
    labels=y_val_hog, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_hog = DataLoader(
    train_dataset_hog, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=False)

val_loader_hog = DataLoader(
    val_dataset_hog, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=False)


model_hog = build_resnet50_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_hog = model_hog.to(device)

In [ ]:
# Create datasets leopard
train_dataset_leopard = ImagesDataset(
    features=X_train_leopard_resampled, 
    labels=y_train_leopard, 
    transform=train_transforms, 
    device=device)

val_dataset_leopard = ImagesDataset(
    features=X_val_leopard, 
    labels=y_val_leopard, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_leopard = DataLoader(
    train_dataset_leopard, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=False)

val_loader_leopard = DataLoader(
    val_dataset_leopard, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=False)


model_leopard = build_resnet50_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_leopard = model_leopard.to(device)

In [ ]:
# Create datasets monkey prosimian
train_dataset_monkey_prosimian = ImagesDataset(
    features=X_train_monkey_prosimian_resampled, 
    labels=y_train_monkey_prosimian_resampled, 
    transform=train_transforms, 
    device=device)

val_dataset_monkey_prosimian = ImagesDataset(
    features=X_val_monkey_prosimian, 
    labels=y_val_monkey_prosimian, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_monkey_prosimian = DataLoader(
    train_dataset_monkey_prosimian, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=False)

val_loader_monkey_prosimian = DataLoader(
    val_dataset_monkey_prosimian, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=False)


model_monkey_prosimian = build_resnet50_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_monkey_prosimian = model_monkey_prosimian.to(device)

In [10]:
# Create datasets rodent
train_dataset_rodent = ImagesDataset(
    features=X_train_rodent_resampled, 
    labels=y_train_rodent_resampled, 
    transform=train_transforms, 
    device=device)

val_dataset_rodent = ImagesDataset(
    features=X_val_rodent, 
    labels=y_val_rodent, 
    transform=val_transforms, 
    device=device)

# Create DataLoaders
train_loader_rodent = DataLoader(
    train_dataset_rodent, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=False)

val_loader_rodent = DataLoader(
    val_dataset_rodent, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=False)



model_rodent = build_resnet50_basic(
    num_classes = config["model"]["num_classes"],
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)
model_rodent = model_rodent.to(device)

In [11]:
model = model_rodent
train_loader = train_loader_rodent
val_loader = val_loader_rodent

In [12]:
wandb.require()
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kseeger7 (kseeger7-georgia-institute-of-technology). Use `wandb login --relogin` to force relogin


True

In [13]:
# ✨ W&B: Initialize a new run to track this model's training
wandb.init(project="wildlife", config=config)

In [14]:
set_seeds(config["experiment"]['seed'])
criterion, optimizer = setup_training(
        model, 
        criterion=config["train"]["criterion"],
        optimizer=config["train"]["optimizer"], 
        lr=config["train"]["lr"], 
        momentum=config["train"]["momentum"])

In [15]:
log_counter = 0
tracking_loss_all = []
train_losses = []  # To store average training loss per epoch
val_losses = []    # To store validation loss per epoch
set_seeds(config["experiment"]['seed'])

start_time = time.time()
for epoch in range(config["train"]["epochs"]):
    #Training step
    avg_train_loss, tracking_loss = train(model, 
                                     train_loader, 
                                     criterion, 
                                     optimizer, 
                                     epoch, config, device=device)
    tracking_loss_all.extend(tracking_loss)  # Append to global list
    train_losses.append(avg_train_loss)  # Store avg training loss
    print(f"Epoch {epoch+1}/{config['train']['epochs']} - Avg Train Loss: {avg_train_loss:.4f}")
    
    # Evaluation step
    eval_metrics = evaluate(model, val_loader, criterion, config, epoch= epoch+1, device=device)
    val_losses.append(eval_metrics["loss"])  # Store validation loss
    print(f"Epoch {epoch+1}/{config['train']['epochs']} - Eval Loss: {eval_metrics['loss']:.4f}, Eval Acc: {eval_metrics['accuracy']:.2f}%")

end_time = time.time()
duration = end_time - start_time
wandb.log({"duration": duration})

Starting training for epoch 1
Epoch [1/10], Step [10/101], Loss: 0.4805
Epoch [1/10], Step [20/101], Loss: 0.6851
Epoch [1/10], Step [30/101], Loss: 0.6288
Epoch [1/10], Step [40/101], Loss: 0.5651
Epoch [1/10], Step [50/101], Loss: 0.7922
Epoch [1/10], Step [60/101], Loss: 0.7120
Epoch [1/10], Step [70/101], Loss: 0.4608
Epoch [1/10], Step [80/101], Loss: 0.6306
Epoch [1/10], Step [90/101], Loss: 0.5256
Epoch [1/10], Step [100/101], Loss: 0.5456
Epoch 1/10 - Avg Train Loss: 0.6128
Evaluation - Loss: 0.5321, Accuracy: 81.23%, Precision: nan, Recall: nan, F1: nan
Epoch 1/10 - Eval Loss: 0.5321, Eval Acc: 81.23%
Starting training for epoch 2
Epoch [2/10], Step [10/101], Loss: 0.5025
Epoch [2/10], Step [20/101], Loss: 0.5387
Epoch [2/10], Step [30/101], Loss: 0.5734
Epoch [2/10], Step [40/101], Loss: 0.5935
Epoch [2/10], Step [50/101], Loss: 0.4780
Epoch [2/10], Step [60/101], Loss: 0.3866
Epoch [2/10], Step [70/101], Loss: 0.3573
Epoch [2/10], Step [80/101], Loss: 0.4760
Epoch [2/10], St

In [16]:
# ✨ W&B: Mark the run as complete (Or wait until the end of notebook)
wandb.finish()


duration,▁
epoch,▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▇▅█▇▇▅▄▇▅▆▅▄▆▅▅▆▄▄▄▆▇█▃▄▅▅▆▅▄▅▆▂▃▃▆▁▄▄▂
duration,781.55842
epoch,10
loss,0.19277


In [17]:
torch.save(model.state_dict(),"../pretrained_models/pretrained_rodent_model.pth")